In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import math
import json

from os.path import join
from pathlib import Path
from tqdm.notebook import tqdm
from itertools import combinations, combinations_with_replacement, product
from utils.style import rc_style
from utils.plotting import aligned_imshow_cbar
from utils.paralell import (agent_name, calculate_beta, isoelastic_utility, 
                            inverse_isoelastic_utility, wealth_change, 
                            shuffle_along_axis, create_gambles, 
                            create_gamble_pairs, create_trial_order, 
                            create_experiment, is_mixed, is_nobrainer, 
                            is_equal_growth, disagreement, bankruptcy_chance, 
                            richness_chance, experiment_duration, 
                            run_simulation, softmax)
mpl.rcParams.update(rc_style)                            

### Settings
- `x_0`: initial wealth
- `x_limit`: wealth limit per agent
- `n_fractals`: number of distinct fractals
- `n_simulations`: number of simulations run in paralell
- `eta_dynamic`: wealth dynamic parameter
- `eta_agent`: agent's risk attitude
- `c`: maximum fractal growth rate; should be adjusted for each dynamic
- `p_threshold`: critial choice probaiblity used to normalize softmax sensitivity across agents. Normalization ensures that all isoelastic agnets would choose the best fractal over the worst fractal with `p_threshold` probability. Values closer to one indicate more sensitive agents and more deterministic choices.
- `wealth_dependency`: decision if wealth dependency effects should be taken into account

In [ ]:
x_0 = 1000
x_limit = 4000
n_fractals = 9
n_trials = 350
p_threshold = 0.9999
agents = [-1, 0, 1]

# Gamble space settings 
n_gamma_thrs = 12
eta_dynamic = 1
c_min = 0.001
c_max = 0.3
n_c = 50

# Other
n_simulations = 1000
wealth_dependency = True

In [ ]:
n_agents = len(agents)

shape_common = (n_gamma_thrs, n_gamma_thrs, n_c)
shape_pair = (math.comb(n_agents, 2), ) + shape_common
shape_single = (n_agents, ) + shape_common

# Statistics
d_p = np.zeros(shape_pair)
d_y = np.zeros(shape_pair)
p_bank = np.zeros(shape_single)
p_rich = np.zeros(shape_single)
avg_len = np.zeros(shape_single)
n_gamble_pairs = np.zeros(shape_common)

c_range = np.linspace(c_min, c_max, n_c)
for low_idx, upp_idx in tqdm(list(product(range(n_gamma_thrs), repeat=2))):
    for c_idx, c in enumerate(c_range):

        gamma_thrs = np.linspace(
            start = -c - c / (2 * (n_fractals - 1)), 
            stop = c + c / (2 * (n_fractals - 1)), 
            num = 2 * n_fractals
            )

        # Create & filter gambles by max average growth rate
        gambles = create_gambles(c, n_fractals=n_fractals)
        gambles = [
            g for g in gambles 
            if np.mean(g) >= gamma_thrs[low_idx] 
            and np.mean(g) <= gamma_thrs[-1 - upp_idx]
        ]

        if len(gambles) == 0:
            break

        # Create & filter gamble pairs exluding no-brainers
        gamble_pairs = create_gamble_pairs(gambles)
        gamble_pairs = [gp for gp in gamble_pairs if not is_nobrainer(gp)]
        n_gamble_pairs[low_idx, upp_idx, c_idx] = len(gamble_pairs)
        
        # Create experiment and randomized trial order
        experiment = create_experiment(gamble_pairs)
        trial_order = create_trial_order(
            n_simulations, 
            experiment.shape[-1], 
            n_trials
        )

        x, y, p = {}, {}, {}
        for agent in agents:

            # Estimate precision
            beta = calculate_beta(
                eta_dynamic=eta_dynamic,
                eta_agent=agent,
                x_0=x_0,
                x_limit=x_limit,
                p_threshold=p_threshold,
                c=c,
            )

            x[agent], p[agent], y[agent] = run_simulation(
                experiment=experiment,
                trial_order=trial_order,
                eta_dynamic=eta_dynamic,
                eta_agent=agent,
                x_0=x_0,
                x_limit=x_limit,
                beta=beta,
                wealth_dependency=wealth_dependency
            )

        # Store statistics
        for i, (a1, a2) in enumerate(combinations(agents, 2)):
            d_p[i, low_idx, upp_idx, c_idx] = disagreement(p[a1], p[a2])        
            d_y[i, low_idx, upp_idx, c_idx] = disagreement(y[a1], y[a2])
        for i, a in enumerate(agents):
            p_bank[i, low_idx, upp_idx, c_idx] = bankruptcy_chance(x[a])
            p_rich[i, low_idx, upp_idx, c_idx] = richness_chance(x[a], x_limit)
            avg_len[i, low_idx, upp_idx, c_idx] = experiment_duration(x[a], x_limit)

In [ ]:
path_output = join("data", f"simulation_{agent_name(eta_dynamic)}")
Path(path_output).mkdir(exist_ok=True)

# Store main data
np.save(join(path_output, "d_p.npy"), d_p)
np.save(join(path_output, "d_y.npy"), d_y)
np.save(join(path_output, "p_rich.npy"), p_rich)
np.save(join(path_output, "p_bank.npy"), p_bank)
np.save(join(path_output, "avg_len.npy"), avg_len)
np.save(join(path_output, "n_gamble_pairs.npy"), n_gamble_pairs)

# Create and store metadata
gamma_thrs_norm = np.linspace(
    start = -1 - 1 / (2 * (n_fractals - 1)), 
    stop = 1 + 1 / (2 * (n_fractals - 1)), 
    num = 2 * n_fractals
)

dimensions = {
    0: list(combinations(agents, 2)),
    1: list(gamma_thrs_norm[:n_gamma_thrs]),
    2: list(gamma_thrs_norm[-n_gamma_thrs:]),
    3: list(c_range)
}

meta = {
    "x_0": x_0,
    "x_limit": x_limit,
    "n_fractals": n_fractals,
    "n_trials": n_trials,
    "p_threshold": p_threshold,
    "agents": agents,
    "n_gamma_thrs": n_gamma_thrs,
    "eta_dynamic": eta_dynamic,
    "c_min": c_min,
    "c_max": c_max,
    "n_c": n_c,
    "n_simulations": n_simulations,
    "wealth_dependency": wealth_dependency,
    "dimensions": dimensions
}

with open(join(path_output, "metadata.json"), "w") as f:
    f.writelines(json.dumps(meta, indent=2))